In [1]:
import torch
import joblib
import os
import time
from utils import *
from tqdm import tqdm_notebook as tqdm
os.listdir('./model')

['net.pkl', 'StandardScaler.pkl']

In [2]:
import numpy as np

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# load model

In [3]:
s = time.time()
ss = joblib.load('./model/StandardScaler.pkl')
net = torch.load('./model/net.pkl')
net.eval()
print('loading time: ',time.time()-s)

loading time:  1.6842656135559082


# load data

In [4]:
df = pd.read_csv('./data/train(add_split_factor).csv',index_col=0).sample(1000).reset_index(drop=True)
print(df.shape)
df.head()

(1000, 436)


,Case Conditions_Feed Rate to C013 (Input),Case Conditions_C7+ in Light End (Input),Case Conditions_Heart Cut Prod. Rate (Input),Case Conditions_C6P- in Heart Cut (Input),Case Conditions_Feed Rate to C013 (Results),Case Conditions_C7+ in Light End (Results),Case Conditions_Heart Cut Prod. Rate (Results),Case Conditions_C6P- in Heart Cut (Results),Case Conditions_C10+ in Heart Cut (Results),Naphtha Properties_Naphtha Feed Rate,...,"Individual Component to Heavy End Split Factor_2,2,3,3-Tetramethylhexane",Individual Component to Heavy End Split Factor_3-Methylnonane,Individual Component to Heavy End Split Factor_tert-Butylcyclohexane,Individual Component to Heavy End Split Factor_n-Decane,"Individual Component to Heavy End Split Factor_1,2-Diethylbenzene",Individual Component to Heavy End Split Factor_n-Undecane,Individual Component to Heavy End Split Factor_n-Pentylbenzene,Individual Component to Heavy End Split Factor_n-Dodecane,Individual Component to Heavy End Split Factor_n-Hexylbenzene,Individual Component to Heavy End Split Factor_n-Tridecane
0,10.0,2.5,100.0,1.0,10.000000,2.499997,99.999985,0.999999,3.792618,120.699997,...,0.380760,0.410276,0.413736,0.428628,0.440890,0.464364,0.471375,0.481158,0.484643,1.0
1,10.0,2.5,90.0,0.3,10.000000,2.499796,89.999908,0.300051,3.811946,101.699997,...,0.357849,0.381945,0.386022,0.398366,0.411111,0.430496,0.437799,0.445717,0.449417,1.0
2,20.0,2.5,90.0,1.0,20.000000,2.499930,90.001297,0.999834,4.554509,108.000008,...,0.378046,0.408109,0.412967,0.427616,0.441895,0.465331,0.473338,0.482981,0.486990,1.0
3,20.0,10.0,100.0,1.0,20.000000,10.000336,99.999695,0.999774,6.903033,125.000008,...,0.345650,0.380054,0.386314,0.401702,0.417925,0.442881,0.451769,0.461932,0.466387,1.0
4,20.0,2.5,100.0,1.5,20.000002,2.499806,99.999825,1.500343,5.521711,121.700005,...,0.368941,0.398351,0.401912,0.416822,0.429315,0.452851,0.460027,0.469849,0.473428,1.0


# define columns

In [5]:
case_col = df.columns[(df.columns.str.contains('Case'))&(df.columns.str.contains('Input'))].tolist()
case_col

['Case Conditions_Feed Rate to C013 (Input)',
 'Case Conditions_C7+ in Light End (Input)',
 'Case Conditions_Heart Cut Prod. Rate (Input)',
 'Case Conditions_C6P- in Heart Cut (Input)']

In [6]:
naphtha_col = df.columns[df.columns.str.contains('Naphtha Properties')].tolist()[5:-1]
print(len(naphtha_col))
naphtha_col[0],naphtha_col[-1]

54


('Naphtha Properties_Oxygen', 'Naphtha Properties_n-Tridecane')

In [7]:
s_col = get_group_col(df,'Individual Component')

# sort s_col
temp = []
for i in [i.split('_')[1] for i in naphtha_col]:
    col1 = 'Individual Component to Light End Split Factor_{}'.format(i)
    col2 = 'Individual Component to Heart Cut Split Factor_{}'.format(i)
    col3 = 'Individual Component to Heavy End Split Factor_{}'.format(i)
    cols = [col1,col2,col3]
    temp += cols
s_col = temp
print(len(s_col))

162


In [8]:
le_col = get_group_col(df,'Light End Product')[3:-1]
hc_col = get_group_col(df,'Heart Cut Product')[4:-1]
he_col = get_group_col(df,'Heavy End Product')[3:-1]

In [9]:
f_col = 'Naphtha Properties_Naphtha Feed Rate.1'

In [10]:
f_le = get_group_col(df,'Light End Product Properties')[1]
f_hc = get_group_col(df,'Heart Cut Product Properties')[1]
f_he = get_group_col(df,'Heavy End Product Properties')[1]
print(f_le)
print(f_hc)
print(f_he)

Light End Product Properties_Light End Prod. Rate.1
Heart Cut Product Properties_Heart Cut Prod. Rate.1
Heavy End Product Properties_Heavy End Product Rate.1


In [11]:
x_col = case_col + naphtha_col
y_col = s_col
print(len(x_col))
print(len(y_col))

58
162


# tensor

In [12]:
X = torch.tensor(ss.transform(df[x_col]),dtype=torch.float).cuda()
y = torch.tensor(df[y_col].values,dtype=torch.float).cuda()

# make split factor prediction 

In [13]:
y_pred_sp = net(X).detach().cpu().numpy()
y_pred_sp = pd.DataFrame(y_pred_sp,columns=y_col)
y_pred_sp

,Individual Component to Light End Split Factor_Oxygen,Individual Component to Heart Cut Split Factor_Oxygen,Individual Component to Heavy End Split Factor_Oxygen,Individual Component to Light End Split Factor_Methane,Individual Component to Heart Cut Split Factor_Methane,Individual Component to Heavy End Split Factor_Methane,Individual Component to Light End Split Factor_Ethane,Individual Component to Heart Cut Split Factor_Ethane,Individual Component to Heavy End Split Factor_Ethane,Individual Component to Light End Split Factor_Propane,...,Individual Component to Heavy End Split Factor_n-Pentylbenzene,Individual Component to Light End Split Factor_n-Dodecane,Individual Component to Heart Cut Split Factor_n-Dodecane,Individual Component to Heavy End Split Factor_n-Dodecane,Individual Component to Light End Split Factor_n-Hexylbenzene,Individual Component to Heart Cut Split Factor_n-Hexylbenzene,Individual Component to Heavy End Split Factor_n-Hexylbenzene,Individual Component to Light End Split Factor_n-Tridecane,Individual Component to Heart Cut Split Factor_n-Tridecane,Individual Component to Heavy End Split Factor_n-Tridecane
0,0.999793,0.000101,0.000106,0.999635,0.000220,0.000145,0.999674,0.000263,0.000063,0.998069,...,0.442171,0.001982,0.536440,0.461578,0.001806,0.516885,0.481309,0.000165,0.000088,0.999747
1,0.999620,0.000161,0.000218,0.999501,0.000302,0.000197,0.999659,0.000281,0.000060,0.995932,...,0.427614,0.004461,0.560313,0.435226,0.002007,0.557210,0.440783,0.000363,0.000169,0.999469
2,0.996150,0.002211,0.001639,0.993468,0.004038,0.002494,0.994673,0.003639,0.001688,0.980226,...,0.441376,0.016465,0.540310,0.443225,0.011045,0.526743,0.462212,0.002738,0.001805,0.995457
3,0.999860,0.000056,0.000084,0.999651,0.000270,0.000079,0.999661,0.000269,0.000070,0.996476,...,0.507911,0.002118,0.475183,0.522699,0.001862,0.468736,0.529401,0.000062,0.000043,0.999895
4,0.999813,0.000087,0.000100,0.999702,0.000196,0.000102,0.999688,0.000244,0.000068,0.998755,...,0.427479,0.001775,0.563427,0.434798,0.001209,0.548940,0.449851,0.000131,0.000075,0.999794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.999560,0.000222,0.000217,0.999448,0.000352,0.000200,0.999679,0.000256,0.000065,0.996633,...,0.401948,0.005686,0.589902,0.404412,0.002250,0.572676,0.425073,0.000423,0.000212,0.999365
996,0.999740,0.000110,0.000150,0.999622,0.000233,0.000144,0.999762,0.000199,0.000039,0.996702,...,0.442158,0.003889,0.543121,0.452990,0.001711,0.540596,0.457693,0.000263,0.000118,0.999619
997,0.999217,0.000359,0.000424,0.997969,0.001381,0.000649,0.998012,0.001426,0.000563,0.991227,...,0.484780,0.005551,0.492297,0.502152,0.004596,0.491635,0.503768,0.000550,0.000368,0.999082
998,0.999621,0.000181,0.000198,0.999823,0.000130,0.000047,0.999801,0.000160,0.000040,0.997710,...,0.459468,0.003789,0.544405,0.451806,0.002453,0.529944,0.467604,0.000151,0.000110,0.999739


In [14]:
y_real_sp = df[y_col]
y_real_sp

,Individual Component to Light End Split Factor_Oxygen,Individual Component to Heart Cut Split Factor_Oxygen,Individual Component to Heavy End Split Factor_Oxygen,Individual Component to Light End Split Factor_Methane,Individual Component to Heart Cut Split Factor_Methane,Individual Component to Heavy End Split Factor_Methane,Individual Component to Light End Split Factor_Ethane,Individual Component to Heart Cut Split Factor_Ethane,Individual Component to Heavy End Split Factor_Ethane,Individual Component to Light End Split Factor_Propane,...,Individual Component to Heavy End Split Factor_n-Pentylbenzene,Individual Component to Light End Split Factor_n-Dodecane,Individual Component to Heart Cut Split Factor_n-Dodecane,Individual Component to Heavy End Split Factor_n-Dodecane,Individual Component to Light End Split Factor_n-Hexylbenzene,Individual Component to Heart Cut Split Factor_n-Hexylbenzene,Individual Component to Heavy End Split Factor_n-Hexylbenzene,Individual Component to Light End Split Factor_n-Tridecane,Individual Component to Heart Cut Split Factor_n-Tridecane,Individual Component to Heavy End Split Factor_n-Tridecane
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.995605,...,0.471375,0.0,0.518842,0.481158,0.0,0.515357,0.484643,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.994070,...,0.437799,0.0,0.554283,0.445717,0.0,0.550583,0.449417,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.989833,...,0.473338,0.0,0.517018,0.482981,0.0,0.513010,0.486990,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.991250,...,0.451769,0.0,0.538068,0.461932,0.0,0.533613,0.466387,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.990262,...,0.460027,0.0,0.530150,0.469849,0.0,0.526572,0.473428,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.993668,...,0.390325,0.0,0.602217,0.397783,0.0,0.598636,0.401364,0.0,0.0,1.0
996,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.994616,...,0.432003,0.0,0.560104,0.439896,0.0,0.556356,0.443644,0.0,0.0,1.0
997,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.990433,...,0.561971,0.0,0.426948,0.573052,0.0,0.422660,0.577340,0.0,0.0,1.0
998,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.995704,...,0.458264,0.0,0.533459,0.466541,0.0,0.529556,0.470444,0.0,0.0,1.0


# use split factor reconstrction output

In [15]:
y_real_o = df[le_col+hc_col+he_col]
y_real_o

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,"Heavy End Product Properties_2,2,3,3-Tetramethylhexane",Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane
0,0.0,0.0,0.0,0.023653,0.035277,0.314350,1.031269,0.011594,2.582455,20.656391,...,4.737848,11.918348,1.504177,2.126353,0.915204,0.435675,0.055282,0.005643,0.011368,0.0
1,0.0,0.0,0.0,0.581659,1.129213,9.310719,16.314816,0.221603,1.918643,17.001333,...,3.880165,9.668570,5.140269,2.522411,3.792951,0.514754,0.237948,0.012113,0.048853,0.0
2,0.0,0.0,0.0,0.367081,0.724705,5.898250,10.385075,0.140636,2.046021,19.573011,...,5.174518,13.030694,3.700563,3.852883,1.658974,0.790427,0.104873,0.011890,0.017983,0.0
3,0.0,0.0,0.0,0.347488,0.687375,5.628091,10.672565,0.131144,2.208220,18.879107,...,5.141209,13.198468,5.265100,14.022647,13.582584,2.923714,0.851054,0.052969,0.168078,0.0
4,0.0,0.0,0.0,0.331544,0.666545,5.403596,9.706414,0.124869,1.177000,22.916021,...,5.252141,13.233355,3.591440,4.087497,2.029320,0.838265,0.126719,0.015531,0.026082,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.502813,0.981624,8.044596,14.531475,0.198171,2.382025,15.713842,...,4.224062,10.538236,4.762571,2.482967,2.629691,0.508748,0.165337,0.011233,0.034003,0.0
996,0.0,0.0,0.0,0.510911,1.014657,8.225545,14.250669,0.204017,1.725411,14.796736,...,3.953165,9.855848,5.035010,2.407876,3.620697,0.490810,0.221932,0.006277,0.044316,0.0
997,0.0,0.0,0.0,0.439412,0.868441,7.054605,14.213878,0.165422,1.364167,19.056156,...,7.037123,17.889013,3.056424,6.827018,4.942161,1.399722,0.306673,0.028429,0.064444,0.0
998,0.0,0.0,0.0,0.444089,0.883215,7.281281,13.253098,0.177434,2.567274,22.881920,...,3.282253,8.163488,1.636493,3.671440,2.260897,0.749914,0.141239,0.011503,0.028999,0.0


In [16]:
y_pred_o = pd.DataFrame(index=y_real_o.index,columns=y_real_o.columns)
y_pred_o

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,"Heavy End Product Properties_2,2,3,3-Tetramethylhexane",Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def reconstrction(df,y_pred_sp):
    y_pred_o = pd.DataFrame(index=df.index,columns=le_col+hc_col+he_col)
    for x in naphtha_col:
        c = x.split('_')[1]
        
        s_le = 'Individual Component to Light End Split Factor_{}'.format(c)
        s_hc = 'Individual Component to Heart Cut Split Factor_{}'.format(c)
        s_he = 'Individual Component to Heavy End Split Factor_{}'.format(c)
        
        o_le = 'Light End Product Properties_{}'.format(c)
        o_hc = 'Heart Cut Product Properties_{}'.format(c)
        o_he = 'Heavy End Product Properties_{}'.format(c)
        
        y_pred_o[o_le] = df[f_col] * df[x] * y_pred_sp[s_le]
        y_pred_o[o_hc] = df[f_col] * df[x] * y_pred_sp[s_hc]
        y_pred_o[o_he] = df[f_col] * df[x] * y_pred_sp[s_he]

    y_pred_o[le_col] = y_pred_o[le_col].values / y_pred_o[le_col].sum(axis=1).values.reshape(-1,1)*100
    y_pred_o[hc_col] = y_pred_o[hc_col].values / y_pred_o[hc_col].sum(axis=1).values.reshape(-1,1)*100
    y_pred_o[he_col] = y_pred_o[he_col].values / y_pred_o[he_col].sum(axis=1).values.reshape(-1,1)*100
    
    return y_pred_o

In [18]:
y_pred_o = reconstrction(df,y_pred_sp)

In [19]:
y_pred_o['SUM'] = y_pred_o.sum(axis=1)
y_real_o['SUM'] = y_real_o.sum(axis=1)

In [20]:
y_pred_o.head()

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane,SUM
0,0.0,0.0,0.0,0.023166,0.033814,0.298955,0.978511,0.010323,2.430059,19.099803,...,11.475473,1.447238,2.146299,0.888795,0.431563,0.052589,0.005490,0.011449,0.0,300.0
1,0.0,0.0,0.0,0.561284,1.063597,8.767213,15.321688,0.211083,1.773998,15.472964,...,9.094395,4.748025,2.339835,3.586253,0.475962,0.218447,0.011117,0.045035,0.0,300.0
2,0.0,0.0,0.0,0.339434,0.681658,5.519552,9.668254,0.133646,1.914865,18.101869,...,12.051969,3.458576,3.628673,1.547066,0.738978,0.100048,0.011163,0.017462,0.0,300.0
3,0.0,0.0,0.0,0.357993,0.709647,5.769566,10.789218,0.133657,2.243158,19.342290,...,13.928892,5.491452,14.927071,14.430660,3.101554,0.911660,0.057108,0.181784,0.0,300.0
4,0.0,0.0,0.0,0.328369,0.657348,5.331062,9.590317,0.125125,1.163636,22.253266,...,12.834410,3.513538,3.941947,2.025044,0.823735,0.122970,0.015009,0.025881,0.0,300.0


In [21]:
y_real_o.head()

,Light End Product Properties_Oxygen,Light End Product Properties_Methane,Light End Product Properties_Ethane,Light End Product Properties_Propane,Light End Product Properties_n-Butane,Light End Product Properties_i-Pentane,Light End Product Properties_n-Pentane,Light End Product Properties_tr2-Pentene,Light End Product Properties_Cyclopentane,Light End Product Properties_3-Methylpentane,...,Heavy End Product Properties_3-Methylnonane,Heavy End Product Properties_tert-Butylcyclohexane,Heavy End Product Properties_n-Decane,"Heavy End Product Properties_1,2-Diethylbenzene",Heavy End Product Properties_n-Undecane,Heavy End Product Properties_n-Pentylbenzene,Heavy End Product Properties_n-Dodecane,Heavy End Product Properties_n-Hexylbenzene,Heavy End Product Properties_n-Tridecane,SUM
0,0.0,0.0,0.0,0.023653,0.035277,0.314350,1.031269,0.011594,2.582455,20.656391,...,11.918348,1.504177,2.126353,0.915204,0.435675,0.055282,0.005643,0.011368,0.0,300.000013
1,0.0,0.0,0.0,0.581659,1.129213,9.310719,16.314816,0.221603,1.918643,17.001333,...,9.668570,5.140269,2.522411,3.792951,0.514754,0.237948,0.012113,0.048853,0.0,299.999990
2,0.0,0.0,0.0,0.367081,0.724705,5.898250,10.385075,0.140636,2.046021,19.573011,...,13.030694,3.700563,3.852883,1.658974,0.790427,0.104873,0.011890,0.017983,0.0,300.000018
3,0.0,0.0,0.0,0.347488,0.687375,5.628091,10.672565,0.131144,2.208220,18.879107,...,13.198468,5.265100,14.022647,13.582584,2.923714,0.851054,0.052969,0.168078,0.0,300.000010
4,0.0,0.0,0.0,0.331544,0.666545,5.403596,9.706414,0.124869,1.177000,22.916021,...,13.233355,3.591440,4.087497,2.029320,0.838265,0.126719,0.015531,0.026082,0.0,299.999983


# metric

In [22]:
y_col = y_real_o.columns.tolist()
y_real = y_real_o
y_pred = y_pred_o
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [23]:
res = pd.DataFrame(index=y_col,columns=['MSE','MAPE'])
for i in y_col:
  res.loc[i,'MSE'] = mean_squared_error(y_real[i],y_pred[i])
  res.loc[i,'MAPE'] = mape(y_real[i],y_pred[i])
res.loc['AVG'] = res.mean(axis=0)
res

,MSE,MAPE
Light End Product Properties_Oxygen,0,NaN
Light End Product Properties_Methane,0,NaN
Light End Product Properties_Ethane,0,NaN
Light End Product Properties_Propane,0.00131207,NaN
Light End Product Properties_n-Butane,0.00385776,NaN
Light End Product Properties_i-Pentane,0.24841,4.5181
Light End Product Properties_n-Pentane,0.62209,4.54834
Light End Product Properties_tr2-Pentene,0.000208705,NaN
Light End Product Properties_Cyclopentane,0.00966749,NaN
Light End Product Properties_3-Methylpentane,1.46176,5.91264
